# v1 code

In [13]:
from itertools import permutations, product, chain, zip_longest
from fractions import Fraction

def solve(digits):
    # 将数字转换为字符串形式
    digits = list(map(str, digits))
    num_len = len(digits)
    
    # 所有数字排列组合
    digit_permutations = sorted(set(permutations(digits)))
    
    # 所有运算符的排列组合
    operator_combinations = list(product('+-*/', repeat=num_len - 1))
    
    # 括号插入位置的组合
    brackets = (
        [()] +  # 无括号的情况
        [(x, y) for x in range(0, 2*num_len-1, 2) for y in range(x+4, 2*num_len+1, 2)]
    )
    
    # 遍历所有数字和运算符组合
    for digit_perm in digit_permutations:
        for ops in operator_combinations:
            # 如果有除法，使用 Fraction 确保精确度
            if '/' in ops:
                expr_digits = [('Fraction(%s)' % d) for d in digit_perm]
            else:
                expr_digits = digit_perm
            
            # 生成基本的表达式（无括号）
            expression = list(chain.from_iterable(zip_longest(expr_digits, ops, fillvalue='')))
            
            # 遍历所有括号的组合情况
            for b in brackets:
                expr_with_brackets = expression[:]
                for insert_index, bracket in zip(b, '()' * (len(b) // 2)):
                    expr_with_brackets.insert(insert_index, bracket)
                
                expr_str = ''.join(expr_with_brackets)
                
                try:
                    # 计算表达式结果
                    result = eval(expr_str)
                except ZeroDivisionError:
                    continue  # 遇到除零错误跳过
                
                # 判断结果是否为24
                if result == 24:
                    if '/' in ops:
                        expr_with_brackets = [
                            (term if not term.startswith('Fraction(') else term[9:-1]) 
                            for term in expr_with_brackets
                        ]
                    ans = ' '.join(expr_with_brackets).rstrip()
                    print(f"Solution found: {ans}")
                    return ans
    
    # 没有找到解决方案
    print(f"No solution found for: {' '.join(digits)}")
    return None

# Test case
solve([8, 1, 3, 2])

Solution found: ( 1 + 3 + 8 ) * 2


'( 1 + 3 + 8 ) * 2'

# v2 code

In [16]:
import tkinter as tk
from tkinter import ttk

# 创建Tkinter主窗口
root = tk.Tk()
root.title("24-Point Game")

# 扑克选项 (A=1, 2-10, J=11, Q=12, K=13)
card_display_values = ['A'] + [str(i) for i in range(2, 11)] + ['J', 'Q', 'K']
card_actual_values = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13}

# 初始化四个下拉菜单变量
card1 = tk.StringVar()
card2 = tk.StringVar()
card3 = tk.StringVar()
card4 = tk.StringVar()

# 定义solve函数的回调
def solve_24_point():
    # 获取选择的卡片值并将其转换为整数列表
    selected_cards = [card_actual_values[card1.get()], card_actual_values[card2.get()], card_actual_values[card3.get()], card_actual_values[card4.get()]]
    
    # 调用solve函数并展示结果
    result = solve(selected_cards)
    if result:
        result_label.config(text=f"Solution: {result}", font=("Arial", 14, "bold"))  # 修改字体样式
    else:
        result_label.config(text="No solution found!", font=("Arial", 14, "bold"))

# 创建并放置四个下拉菜单
ttk.Label(root, text="Card 1").grid(column=0, row=0, padx=10, pady=10)
dropdown1 = ttk.Combobox(root, textvariable=card1, values=card_display_values, state='readonly')
dropdown1.grid(column=1, row=0, padx=10, pady=10)

ttk.Label(root, text="Card 2").grid(column=0, row=1, padx=10, pady=10)
dropdown2 = ttk.Combobox(root, textvariable=card2, values=card_display_values, state='readonly')
dropdown2.grid(column=1, row=1, padx=10, pady=10)

ttk.Label(root, text="Card 3").grid(column=0, row=2, padx=10, pady=10)
dropdown3 = ttk.Combobox(root, textvariable=card3, values=card_display_values, state='readonly')
dropdown3.grid(column=1, row=2, padx=10, pady=10)

ttk.Label(root, text="Card 4").grid(column=0, row=3, padx=10, pady=10)
dropdown4 = ttk.Combobox(root, textvariable=card4, values=card_display_values, state='readonly')
dropdown4.grid(column=1, row=3, padx=10, pady=10)

# 创建并放置"Submit"按钮
submit_button = ttk.Button(root, text="Submit", command=solve_24_point)
submit_button.grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# 创建显示结果的标签
result_label = ttk.Label(root, text="")
result_label.grid(column=0, row=5, columnspan=2, padx=10, pady=10)

# 运行Tkinter主循环
root.mainloop()

# v3 code

In [18]:
import cv2
import pytesseract
from PIL import Image

def preprocess_image(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use adaptive thresholding to create a binary image
    binary = cv2.adaptiveThreshold(
        blurred, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 11, 2
    )

    # Save the preprocessed image for reference
    cv2.imwrite('preprocessed_image.png', binary)

    return binary

def recognize_card_numbers(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)

    # Convert the preprocessed image to PIL format
    pil_image = Image.fromarray(preprocessed_image)

    # Use pytesseract to detect the bounding boxes of the characters
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789'
    detected_boxes = pytesseract.image_to_boxes(pil_image, config=custom_config)

    # Store recognized numbers and their positions
    recognized_numbers = []

    # Parse detected boxes
    for line in detected_boxes.splitlines():
        char, x, y, w, h, _ = line.split()
        if char.isdigit():
            recognized_numbers.append((char, int(x)))

    # Sort recognized numbers by their x-position (left to right)
    recognized_numbers = sorted(recognized_numbers, key=lambda x: x[1])

    # Extract only the digits from the sorted list
    recognized_digits = [num[0] for num in recognized_numbers]

    # Return the first 4 recognized digits, or as many as are available
    return recognized_digits[:4]

In [19]:
# Example usage
if __name__ == "__main__":
    image_path = 'test2.png'  # Replace with your image file path
    card_numbers = recognize_card_numbers(image_path)
    print(f"Recognized Card Numbers: {card_numbers}")

Recognized Card Numbers: ['3', '1', '7', '7']


# Final code

# Import necessary libraries

In [12]:
import tkinter as tk
from tkinter import ttk
from itertools import permutations, product, chain, zip_longest
from fractions import Fraction
import time

## Codes

In [16]:
# Create the Tkinter main window
root = tk.Tk()
root.title("24 Point Referee")

# Initial 13 cards (A=1, 2-10, J=11, Q=12, K=13)
card_display_values = ['A'] + [str(i) for i in range(2, 11)] + ['J', 'Q', 'K']
card_actual_values = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 
                      8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13}


# Initialize variables for dropdown menus
card1, card2, card3, card4 = tk.StringVar(), tk.StringVar(), tk.StringVar(), tk.StringVar()

# Function to find a solution for 24-point game
def solve(digits):
    # Convert numbers to strings for easier handling
    digits = list(map(str, digits))
    num_len = len(digits)
    
    # Generate all permutations of the digits
    digit_permutations = sorted(set(permutations(digits)))
    
    # Generate all combinations of operators (+, -, *, /)
    operator_combinations = list(product('+-*/', repeat=num_len - 1))
    
    # Brackets positions for different combinations
    brackets = (
        [()] +  # No brackets
        [(x, y) for x in range(0, 2*num_len-1, 2) for y in range(x+4, 2*num_len+1, 2)]
    )
    
    # Iterate through permutations and operator combinations
    for digit_perm in digit_permutations:
        for ops in operator_combinations:
            # Use fractions for division to avoid floating-point errors
            expr_digits = [('Fraction(%s)' % d) if '/' in ops else d for d in digit_perm]
            # Create a basic expression without brackets
            expression = list(chain.from_iterable(zip_longest(expr_digits, ops, fillvalue='')))
            
            # Add brackets and evaluate each expression
            for b in brackets:
                expr_with_brackets = expression[:]
                for insert_index, bracket in zip(b, '()' * (len(b) // 2)):
                    expr_with_brackets.insert(insert_index, bracket)
                expr_str = ''.join(expr_with_brackets)
                
                try:
                    # Evaluate the expression
                    result = eval(expr_str)
                except ZeroDivisionError:
                    continue
                
                # Check if the result equals 24
                if result == 24:
                    # Format the output for readability
                    if '/' in ops:
                        expr_with_brackets = [
                            term if not term.startswith('Fraction(') else term[9:-1] 
                            for term in expr_with_brackets
                        ]
                    return ' '.join(expr_with_brackets).rstrip()
    return None

# Callback function for the "Submit" button
def solve_24_point():
    # Convert selected card values to integers
    selected_cards = [card_actual_values[card1.get()], card_actual_values[card2.get()],
                      card_actual_values[card3.get()], card_actual_values[card4.get()]]
    
    # Call the solve function and display the result
    result = solve(selected_cards)
    
    # Change background color and display result with animation
    if result:
        result_label.config(
            text="Calculating...",
            font=("Arial", 14, "bold"),
            background="yellow"
        )
        root.update()
        time.sleep(0.5)  # Simulate a delay to show the calculation process
        result_label.config(
            text=f"Solution: {result}",
            background="lightgreen"
        )
    else:
        result_label.config(
            text="Calculating...",
            font=("Arial", 14, "bold"),
            background="yellow"
        )
        root.update()
        time.sleep(0.5)  # Simulate a delay to show the calculation process
        result_label.config(
            text="No solution found!",
            background="lightcoral"
        )

# Create and place the dropdowns for selecting cards
for idx, card_var in enumerate([card1, card2, card3, card4], start=1):
    ttk.Label(root, text=f"Card {idx}").grid(column=0, row=idx-1, padx=10, pady=5)
    dropdown = ttk.Combobox(root, textvariable=card_var, values=card_display_values, state='readonly')
    dropdown.grid(column=1, row=idx-1, padx=10, pady=5)
    dropdown.set(card_display_values[0])  # Default selection

# Create the "Submit" button
submit_button = ttk.Button(root, text="Calculate", command=solve_24_point)
submit_button.grid(column=0, row=4, columnspan=2, pady=10)

# Label to display the result with a dynamic background color
result_label = tk.Label(root, text="", wraplength=300, font=("Arial", 14))
result_label.grid(column=0, row=5, columnspan=2, pady=10)

# Adjust window size for better layout
root.geometry("300x250")
root.resizable(False, False)
root.mainloop()

pyinstaller --onefile --windowed --icon=./Bambu_Studio.icns 24_point_game.py

## Video

我的项目是“24 Point Referee”，可以快速在计算24点的游戏中判断是否有解决方案。这是一个基于python的Application，所以有两种运行方式。第一种是通过python代码直接运行，可以在notebook或者PyCharm中调试或者运行，我的大部份工作都是没有可视化界面的调试，为了便于其他人操作，我封装成了一个电脑应用程序，这是第二种运行方式，当作电脑软件来使用。点击进入后需要手动输入4张扑克牌的号码，其中J、Q、K代表数字11、12和13。

在可视化的时候，我选择了三种不同的颜色表示程序的进度，黄色表示正在计算中，绿色表示求出了结果，红色表示没有结果，这样更加醒目。

我的代码迭代更新了多次，先解决了求解问题，我参考了网络上的一些思路，用循环的方式将所有计算方式排列组合，将输入的4个数字转成int的数据类型就可以开始计算了。下面的循环就是罗列出所有可能的排列方式。
代码的第二部份是和可视化界面相关的，首先用字典的数据格式将扑克牌的序号映射到数字，然后初始化应用的界面，这里自定义了应用名称、界面尺寸信息。将可视化界面单独封装成solve_24_point的def，这样方便调用。最后是tkinter需要的模版代码，将各个label、选择框和文本框排布在界面上。这个应用就可以开始运行了。

额外的一个内容是将应用封装成app，用到了pyinstall工具，在命令行中一行代码就可以搞定。

- My project is called the 24 Point Referee. It’s a Python app that checks if four cards can make 24. You pick four cards, where Jack, Queen, and King count as 11, 12, and 13. The app shows its progress in three colors: yellow means ‘calculating,’ green means ‘solution found,’ and red means ‘no solution,’ so it’s clear to follow.

- Now, let’s look at the code that makes this work. First, I set up a dictionary to convert card names to numbers. Then, in the main part, the solve function creates every possible order of the four cards and matches them with different math operations—like add, subtract, multiply, and divide. This way, we can try every possible way to make 24. I use Python functions to make these combinations and run through each one with a loop.

- Inside the loop, the code builds expressions by putting numbers and operators together. I add brackets where needed to keep the math order right. Then, each expression is checked to see if it equals 24. If one does, it’s shown as the solution; if not, it keeps trying other options. Using Python’s eval function helps avoid small errors, especially with dividing.

- For the app’s look, I used tkinter to make dropdowns for picking cards and a button to run the game. The function solve_24_point pulls it all together, so when you click ‘Calculate,’ the app runs, and results are shown in color for easy reading.

- Finally, I used a tool called PyInstaller to make this a standalone app that others can use on their computers. Thanks for watching!

Jupyter Notebook was my initial choice for coding, as it allowed me to run code line-by-line which proved useful. This step-by-step method helped me understand each component better. Later, I switched to PyCharm. The transition wasn’t easy because of the setup time, but the interface’s debugging tools and variable tracking helped me gain a clearer view of variable types and values. I plan to explore PyCharm’s debugging tools further to identify bugs more easily.